# Geospatial Data Analysis using DuckDB
### Simplifying Spatial Data Analysis with DuckDB’s Spatial Extensions

In the ever-evolving world of data analysis, geospatial data has emerged as a critical component across various industries, from urban planning and environmental monitoring to logistics and retail. However, analyzing spatial data often comes with its own set of challenges, including complex workflows, specialized tools, and performance bottlenecks.

Enter DuckDB, a lightweight, fast, and embeddable analytical database designed to simplify data analysis tasks. With its spatial extensions, DuckDB is now poised to revolutionize how we handle geospatial data, making it more accessible and efficient for analysts and developers alike.

This article explores how DuckDB’s spatial capabilities can streamline geospatial data analysis, offering a powerful yet user-friendly alternative to traditional GIS tools. Whether you’re a data scientist, GIS professional, or a developer looking to integrate spatial analysis into your applications, DuckDB’s spatial extensions provide a compelling solution to simplify and accelerate your workflows.

### Our Sample Dataset
For this article, I will use the dataset from https://www.kaggle.com/datasets/shengjunlim/singapore-mrt-lrt-stations-with-coordinates?resource=download. This dataset contains a list of MRT and LRT stations in Singapore. The following shows the first five rows of the CSV file (MRT Stations.csv):

This dataset includes the latitude and longitude of each MRT and LRT station, along with a field containing a geometry object. For example, POINT (103.9032524667383 1.319778951553637)represents a geometry object in Well-Known Text (WKT) format, which is commonly used for spatial data.

### Importing the CSV file into DuckDB
The first step to working with this CSV file is to load it into a DuckDB dataset. To do so, you need to install the duckdb package:

!pip install duckdb

Once this is done, you can now create an in-memory copy of a DuckDB database:

